## Constants

In [85]:
import os

FILE_COUNT = 50   # TODO: change to 1000
PLAYLISTS_PER_FILE = 1000

DIR_DATA_RAW = os.path.join("..", "data", "raw")
DIR_DATA_PROCESSED = os.path.join("..", "data", "processed")

PLAYLIST_COUNT = FILE_COUNT * PLAYLISTS_PER_FILE

Merge the data extracted from the Spotify API (csv file) with that of the Spotify Million Playlist Dataset (pickle file)

In [86]:
import os
import pandas as pd

path_audio_features = os.path.join(DIR_DATA_RAW, "tracks", "track_audio_features.csv")
audio_features = pd.read_csv(path_audio_features).set_index("id")

display(audio_features)

,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
id,,,,,,,,,,,,,
3zyYVItCMCjFzBHTyjrxPK,79.227,8.0,1.0,-3.406,0.466,0.856,0.3180,0.257000,0.000000,0.6750,0.531,292223.0,4.0
4jSy0HTIoC9yiwZ8OVyTCW,164.207,1.0,1.0,-7.972,0.510,0.849,0.1190,0.000546,0.000748,0.6890,0.887,315067.0,4.0
4zyqBSUFNkJ20mw1FB68gt,83.947,4.0,0.0,-22.867,0.308,0.114,0.0321,0.958000,0.902000,0.0853,0.303,350906.0,4.0
63B3TtwUzOoJoe3unMteVa,93.696,1.0,0.0,-4.166,0.660,0.943,0.2770,0.129000,0.000000,0.5570,0.599,210733.0,4.0
7y9iMe8SOB6z3NoHE2OfXl,118.384,0.0,1.0,-3.539,0.675,0.751,0.0296,0.060400,0.000000,0.0893,0.612,181279.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0s9iofqbQdsmWZQJ1TZh1c,118.061,7.0,1.0,-13.766,0.550,0.351,0.0443,0.531000,0.006020,0.1500,0.237,162826.0,4.0
5Ozvg38CnCQKmPP8tgSRA6,96.520,0.0,1.0,-4.130,0.609,0.742,0.0477,0.001310,0.000002,0.0725,0.531,256227.0,4.0
2yWsZpLZNmap92YkGkzyDq,179.985,6.0,1.0,-5.899,0.373,0.584,0.0272,0.004810,0.449000,0.1630,0.403,214667.0,3.0


Create a function to compute relevant metrics about a playlist's audio features.

In [87]:
def get_playlist_metrics(df: pd.core.frame.DataFrame):
    def get_statistic(f) -> pd.core.frame.DataFrame:
        return df.apply(f).to_frame().transpose()

    assert isinstance(df, pd.core.frame.DataFrame)
    result = pd.concat({"min" : get_statistic(pd.Series.min),
                        "q1" : get_statistic(lambda x: x.quantile(0.25)),
                        "mean" : get_statistic(pd.Series.mean),
                        "median" : get_statistic(pd.Series.median),
                        "q3" : get_statistic(lambda x: x.quantile(0.75)),
                        "max" : get_statistic(pd.Series.max),
                        "standard deviation" : get_statistic(pd.Series.std),
                        "variance" : get_statistic(pd.Series.var)}, axis=1)
    
    result.columns = result.columns.swaplevel(0, 1)
    result.sort_index(axis=1, level=0, inplace=True)
    return result

Combine extracted and calculated data into a playlist frame

In [92]:
import pickle

playlist_list_path = os.path.join(DIR_DATA_PROCESSED, "playlists" + str(PLAYLIST_COUNT) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlists_metrics = pd.DataFrame()


for i, playlist in enumerate(playlist_data[5:100]):
    playlist_features = pd.DataFrame(columns = audio_features.columns.tolist())

    for track_id in playlist["track_ids"]:
        if track_id not in audio_features.index: # TODO: handle cases when track id has no audio features
            continue

        track_features = pd.DataFrame(audio_features.loc[track_id]).T # .loc gives series, so transpose is needed

        playlist_features = pd.concat([playlist_features if not playlist_features.empty else None, 
                                       track_features], axis=0)
        
    playlist_info = pd.DataFrame(data={k:[v] for k,v in playlist.items() if k != "track_ids"})
    playlist_info.columns = pd.MultiIndex.from_product([['metadata'], playlist_info.columns])
    
    playlist_metrics = pd.concat([playlist_info, get_playlist_metrics(playlist_features)], axis=1)
    playlists_metrics = pd.concat([playlists_metrics, playlist_metrics], ignore_index=True)

display(playlist_info)
display(playlists_metrics)

metadata                                                                     \
      name collaborative  pid modified_at num_tracks num_albums num_followers   
0   tears.         false  391  1509148800         68         56             3   

                         
  num_edits num_artists  
0        18          47

metadata                                                       \
              name collaborative  pid modified_at num_tracks num_albums   
0           run it         false   25  1488412800         41         38   
1             Yeet         false   28  1509062400         81         57   
2             2020         false   40  1490659200         85         80   
3    Road Trippin'         false   42  1490572800        158        131   
4     taylor swift         false   44  1430611200         33         29   
..             ...           ...  ...         ...        ...        ...   
90            spin         false  382  1505347200         14         14   
91    country 2017         false  383  1505606400         31         30   
92  *** good stuff         false  384  1507593600         46         43   
93              mb         false  389  1509062400        174        161   
94          tears.         false  391  1509148800         68         56   

                                       acousticness  ... time_signature  \
   num_followers num_edits num_artists          max  ...       variance   
0              2         8          31       0.2350  ...            NaN   
1              7        26          37       0.6480  ...       0.083333   
2              6        27          71       0.2140  ...       0.000000   
3              2        51         106       0.4740  ...       0.000000   
4              3        18          21       0.6010  ...       0.500000   
..           ...       ...         ...          ...  ...            ...   
90             2         5          13       0.0834  ...            NaN   
91             2        17          29       0.0303  ...            NaN   
92             2        18          37       0.6790  ...       0.166667   
93             2        81         140       0.9450  ...       0.222222   
94             3        18          47       0.9500  ...       0.200000   

   valence                                                                 \
       max      mean  median     min       q1       q3 standard deviation   
0    0.124  0.124000  0.1240  0.1240  0.12400  0.12400                NaN   
1    0.813  0.479658  0.4665  0.0859  0.40025  0.61325           0.196283   
2    0.564  0.431667  0.5350  0.1960  0.36550  0.54950           0.204608   
3    0.851  0.506600  0.4705  0.1960  0.37375  0.58900           0.197884   
4    0.595  0.430800  0.5720  0.1970  0.19900  0.59100           0.212695   
..     ...       ...     ...     ...      ...      ...                ...   
90   0.678  0.678000  0.6780  0.6780  0.67800  0.67800                NaN   
91   0.828  0.828000  0.8280  0.8280  0.82800  0.82800                NaN   
92   0.572  0.364167  0.3095  0.2270  0.23550  0.49450           0.156654   
93   0.707  0.428300  0.4535  0.1090  0.25675  0.58975           0.214304   
94   0.289  0.170400  0.1390  0.1250  0.12500  0.17400           0.069259   

                metadata  
    variance description  
0        NaN         NaN  
1   0.038527         NaN  
2   0.041864         NaN  
3   0.039158         NaN  
4   0.045239         NaN  
..       ...         ...  
90       NaN         NaN  
91       NaN         NaN  
92  0.024541         NaN  
93  0.045926         NaN  
94  0.004797         NaN  

[95 rows x 114 columns]